In [11]:
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.linalg import block_diag

In [12]:
# extract impedance matrix from loaded network file
def impedance_matrix(filename):
    net = pp.converter.from_mpc(filename, f_hz=60)

    # compute the admittance matrix
    pp.runpp(net, numda=False, max_iteration=10)

    matrix_Y = np.array(net._ppc["internal"]["Ybus"].todense())

    # Based on Bolognani's paper, we need to exclude slack bus to get Z
    matrix_Y = matrix_Y[1:, :]
    matrix_Y = matrix_Y[:, 1:]
    matrix_Z = np.linalg.inv(matrix_Y)

    matrix_XX = np.imag(matrix_Z) * 121
    matrix_RR = np.real(matrix_Z) * 121

    return matrix_RR, matrix_XX

In [13]:
filename = 'case34sa_mod.mat'

matrix_R, matrix_X = impedance_matrix(filename)

/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd], sort=False)
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup["element"].loc[~is_line] = idx_trafo


In [18]:
n = matrix_R.shape[0]  # number of nodes
m = 2 * n

A = np.eye(n)
#A = np.zeros((n,n)) # menoryless system
B = np.hstack([matrix_R, matrix_X])
Q = 10 * np.eye(n) # cost of safety 
R = np.eye(m) # cost of u
QR = block_diag(Q, R)

AB = np.hstack([A, B])

sigma = 100000000 # we just need to make sure it's sufficienly large 100000000

W = np.eye(n) # adjust it based on w


# Define the variable
X = cp.Variable((n+m, n+m), symmetric=True)

# Define the objective
objective = cp.Minimize(0)

# Define the constraints
constraints = [X >> 0]  # X is symmetric positive semidefinite
constraints += [cp.trace(X) <= sigma]
transient = AB @ X @ AB.T
for i in range(n):
    for j in range(n):
        constraints += [X[i, j] == (transient[i, j] + W[i, j])]

# Form the problem
prob = cp.Problem(objective, constraints)

# Solve the problem using MOSEK
prob.solve(solver=cp.MOSEK)

# Print the results
print("The optimal value is:", prob.value)
print("The optimal X is:")
print(X.value)


The optimal value is: 0.0
The optimal X is:
[[ 978773.18226764   33882.18326552   35150.54879008 ...   -4533.94954624
    -4569.9179809    -4583.70978317]
 [  33882.18326552 1018538.3207536    63433.73598621 ...   -8775.21387312
    -8838.39474893   -8863.48653473]
 [  35150.54879008   63433.73598621 1043116.60829245 ...   -8058.63029375
    -8080.37846931   -8093.27594376]
 ...
 [  -4533.94954624   -8775.21387312   -8058.63029375 ...  953800.56819658
    -4670.71166603   -4689.76600084]
 [  -4569.9179809    -8838.39474893   -8080.37846931 ...   -4670.71166603
   953642.35399976   -4777.79954665]
 [  -4583.70978317   -8863.48653473   -8093.27594376 ...   -4689.76600084
    -4777.79954665  953583.720149  ]]


In [151]:
X1 = X.value # W = I, vv= 10000 obj = 0
print(X1)

[[13.01964143 -3.01937397  0.28433964 ...  0.25977541  0.25963054
   0.25958127]
 [-3.01937397 13.99131156 -1.75675901 ... -0.75342589 -0.75360559
  -0.75365285]
 [ 0.28433964 -1.75675901 11.68659993 ...  0.144875    0.14463717
   0.14452448]
 ...
 [ 0.25977541 -0.75342589  0.144875   ...  7.06259501 -0.03767592
   0.03882027]
 [ 0.25963054 -0.75360559  0.14463717 ... -0.03767592  7.11505284
  -0.07213811]
 [ 0.25958127 -0.75365285  0.14452448 ...  0.03882027 -0.07213811
   7.03956338]]


In [168]:
X3 = X.value # W = I,  v = 1000 obj: trace
print(X3)

[[ 3.11276588 -1.18473667 -0.01750089 ...  0.28710359  0.28708378
   0.28708732]
 [-1.18473667  3.34810167 -0.75611548 ... -0.74580437 -0.74580814
  -0.74584705]
 [-0.01750089 -0.75611548  2.51356018 ...  0.19881252  0.1987449
   0.1987216 ]
 ...
 [ 0.28710359 -0.74580437  0.19881252 ...  0.82179439  0.04449727
   0.32973226]
 [ 0.28708378 -0.74580814  0.1987449  ...  0.04449727  1.03435432
  -0.10016202]
 [ 0.28708732 -0.74584705  0.1987216  ...  0.32973226 -0.10016202
   0.74778247]]


In [147]:
X4 = X.value # W = I, v = 100000000, obj = 0
print(X4)

[[ 978773.18226764   33882.18326552   35150.54879008 ...   -4533.94954624
    -4569.9179809    -4583.70978317]
 [  33882.18326552 1018538.3207536    63433.73598621 ...   -8775.21387312
    -8838.39474893   -8863.48653473]
 [  35150.54879008   63433.73598621 1043116.60829245 ...   -8058.63029375
    -8080.37846931   -8093.27594376]
 ...
 [  -4533.94954624   -8775.21387312   -8058.63029375 ...  953800.56819658
    -4670.71166603   -4689.76600084]
 [  -4569.9179809    -8838.39474893   -8080.37846931 ...   -4670.71166603
   953642.35399976   -4777.79954665]
 [  -4583.70978317   -8863.48653473   -8093.27594376 ...   -4689.76600084
    -4777.79954665  953583.720149  ]]


In [ ]:
X5 = X.value # W = I, v = 100000000, obj = trace
# solver failed

In [174]:
X6 = X.value # W = 0.01 I, v = 100000000, obj = 0
print(X6)

[[ 977528.67128486   33842.01186304   35102.07768644 ...   -4580.25076324
    -4612.46455244   -4619.36127038]
 [  33842.01186304 1017258.45335941   63352.37600316 ...   -8871.10853828
    -8920.55504522   -8936.09499747]
 [  35102.07768644   63352.37600316 1041790.51352389 ...   -8218.01059125
    -8218.51236916   -8210.92191988]
 ...
 [  -4580.25076324   -8871.10853828   -8218.01059125 ...  953020.66985922
    -3966.28060116   -4164.23808213]
 [  -4612.46455244   -8920.55504522   -8218.51236916 ...   -3966.28060116
   952730.10942683   -4152.91870243]
 [  -4619.36127038   -8936.09499747   -8210.92191988 ...   -4164.23808213
    -4152.91870243  953241.16111019]]


In [16]:
X_value = X.value

X1 = np.ones((m, n))
X2 = np.ones((n,n))

for i in range(m):
    for j in range(n):
        X1[i, j] = X_value[i+n, j]

for i in range(n):
    for j in range(n):
        X2[i, j] = X_value[i, j]

KK = X1 @ np.linalg.inv(X2)

print(KK)

# np.savetxt('Controller_K.csv', KK, delimiter=',')
#np.save('Controller_K.npy', KK)
# validate this KK is stabilzing? or it has been proved in Xinyi's paper

[[-3.05571658e-16  2.65046707e-16 -3.03012932e-16 ... -6.05408237e-16
   1.10697422e-15 -1.05716924e-15]
 [ 7.57645765e-16  3.39311838e-16 -1.52068838e-15 ... -2.54765012e-18
  -1.88104557e-15  1.13814932e-15]
 [-2.10149911e-16 -1.40338838e-16 -2.25691086e-16 ...  4.90498483e-16
  -3.53422074e-15  2.41938182e-15]
 ...
 [ 1.46278330e-16  1.35811890e-16 -2.28028911e-16 ... -5.73384466e-16
   4.89886767e-16  9.85579784e-17]
 [-4.87478288e-17 -1.55621323e-17 -8.66021584e-17 ...  1.28916570e-16
  -1.37928158e-16  4.35689593e-17]
 [-1.56499850e-16 -1.15081039e-16  2.89032355e-16 ...  6.37142956e-16
  -2.68116540e-16 -1.89684887e-16]]


In [17]:
xx = np.ones((33, 1)) * 0.1  # we need to change x to x-x_desired

rho = A + B @ KK

# Compute the eigenvalues
eigenvalues = np.linalg.eigvals(rho)

# Compute the spectral radius
spectral_radius = max(eigenvalues)

print(eigenvalues)

[-1.14610751e-15+1.15228729e-15j -1.14610751e-15-1.15228729e-15j
  7.07320948e-17+7.81380952e-16j  7.07320948e-17-7.81380952e-16j
  6.91952399e-16+0.00000000e+00j  3.40609517e-16+4.81504119e-16j
  3.40609517e-16-4.81504119e-16j -4.90738197e-16+1.43173911e-16j
 -4.90738197e-16-1.43173911e-16j -3.85384989e-16+3.29087254e-16j
 -3.85384989e-16-3.29087254e-16j -1.89030042e-16+4.32696080e-16j
 -1.89030042e-16-4.32696080e-16j -3.32286712e-16+0.00000000e+00j
 -6.85759045e-17+3.10911486e-16j -6.85759045e-17-3.10911486e-16j
 -1.96190332e-16+2.34819313e-16j -1.96190332e-16-2.34819313e-16j
  1.09447020e-16+2.00538061e-16j  1.09447020e-16-2.00538061e-16j
  2.03389802e-16+0.00000000e+00j  2.69246670e-17+1.71087681e-16j
  2.69246670e-17-1.71087681e-16j  1.06995568e-16+0.00000000e+00j
 -8.30716139e-17+6.86955019e-17j -8.30716139e-17-6.86955019e-17j
 -1.12635827e-16+0.00000000e+00j  3.70922461e-17+3.85682591e-17j
  3.70922461e-17-3.85682591e-17j -1.25682335e-17+2.14504805e-17j
 -1.25682335e-17-2.145048

### Validate controller K is strongly stable for not only real (A, B), but also estimate ($\bar{A}, \bar{B}$)

In [157]:
# add different levels of noise
matrix_R1 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))
matrix_X1 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))
matrix_R2 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))
matrix_X2 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))
matrix_R3 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))
matrix_X3 = np.ones((matrix_R.shape[0], matrix_R.shape[1]))


for i in range(matrix_R.shape[0]):
    for j in range(matrix_X.shape[1]):
        matrix_R1[i, j] = matrix_R[i, j] + np.random.normal(0, 0.05 * matrix_R[i, j])
        matrix_X1[i, j] = matrix_X[i, j] + np.random.normal(0, 0.05 * matrix_X[i, j])
        matrix_R2[i, j] = matrix_R[i, j] + np.random.normal(0, 0.2 * matrix_R[i, j])
        matrix_X2[i, j] = matrix_X[i, j] + np.random.normal(0, 0.2 * matrix_X[i, j])
        matrix_R3[i, j] = matrix_R[i, j] + np.random.normal(0, 0.5 * matrix_R[i, j])
        matrix_X3[i, j] = matrix_X[i, j] + np.random.normal(0, 0.5 * matrix_X[i, j])


norm_new = np.linalg.norm(matrix_X2 - matrix_X, 'fro')

print(norm_new)

1.548228244164559


In [182]:
# line reconfiguration
filename1 = 'case34sa_mod1.mat'
filename2 = 'case34sa_mod2.mat'
filename3 = 'case34sa_mod3.mat'

matrix_Rl1, matrix_Xl1 = impedance_matrix(filename1)
matrix_Rl2, matrix_Xl2 = impedance_matrix(filename2)
matrix_Rl3, matrix_Xl3 = impedance_matrix(filename3)

B_new = np.hstack([matrix_R1, matrix_X1])

rho = A + B_new @ (KK) # max suffering error 0.03 guassian, but switch three lines is OK

# Compute the eigenvalues
eigenvalues = np.linalg.eigvals(rho)

# Compute the spectral radius
spectral_radius = max(abs(eigenvalues))

print(spectral_radius)


1.0057268793585217


/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd], sort=False)
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup["element"].loc[~is_line] = idx_trafo
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation wit

In [183]:
# test singular value for perturbation analysis
U, S, Vt = np.linalg.svd(KK)

sigma_min = min(S)

print(sigma_min)

0.01118561427059216
